In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint

import ceci
import h5py

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import Image
from astropy.table import Table

import re
import sacc

Make sure to change your path in the next cell that leads to your TXPipe directory. See examples for IN2P3 and NERSC below.

In [3]:
# user specific paths -- IN2P3 example
#my_txpipe_dir = "/sps/lsst/users/ebarroso/TXPipe/"
my_txpipe_dir = "/global/homes/e/edujb/TXPipe/"
import sys
sys.path.insert(0, my_txpipe_dir) 
# user specific paths -- NERSC example
# my_txpipe_dir = "/pscratch/sd/a/avestruz/TXPipe"

#os.chdir(my_txpipe_dir)

import txpipe

___

In [4]:
step1 = txpipe.TXSourceSelectorMetadetect.make_stage(
    # This file is the input metadetect shear catalog
    shear_catalog=f"{my_txpipe_dir}/data/example/inputs/metadetect_shear_catalog.hdf5",
    # This is an input training set for the tomographic selection
    calibration_table=f"{my_txpipe_dir}data/example/inputs/sample_cosmodc2_w10year_errors.dat",

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/examples/metadetect/config.yml",

    # This is the output file for this stage
    shear_tomography_catalog=f"{my_txpipe_dir}/data/example/outputs_metadetect/shear_tomography_catalog.hdf5"
)

In [5]:
step1.run()
step1.finalize()

Process 0 running selection for rows 0-82,200


In [6]:
step2 = txpipe.CLClusterBinningRedshiftRichness.make_stage(
    # This file is the input metadetect shear catalog
    cluster_catalog=f"{my_txpipe_dir}data/example/inputs/cluster_catalog.hdf5",
    # This is an input training set for the tomographic selection

    # This contains all the options for this stage. You can override them here
    # manually too.
    config=f"{my_txpipe_dir}/notebooks/cluster_counts/gt_treecor_test/config-1deg2-CL.yml",

    # This is the output file for this stage
    cluster_catalog_tomography=f"{my_txpipe_dir}/data/example/output_binning/cluster_catalog_tomography.hdf5"
)



In [7]:
step2.run()
step2.finalize()

In [8]:
step3 = txpipe.TXShearCalibration.make_stage(
    # This file is the input metadetect shear catalog
    shear_catalog=f"{my_txpipe_dir}/data/example/inputs/metadetect_shear_catalog.hdf5",
    shear_tomography_catalog=f"{my_txpipe_dir}/data/example/outputs_metadetect/shear_tomography_catalog.hdf5",
    binned_shear_catalog=f"{my_txpipe_dir}data/example/output_shearcalib/binned_shear_catalog.hdf5",
)

In [9]:
step3.run()
step3.finalize()

Copying extra columns:  []
Copying magnitude:  ['mag_i', 'mag_r', 'mag_z', 'mag_err_i', 'mag_err_r', 'mag_err_z']
Using calibration method: MetaDetectCalibrator
Reading these columns from the 'shear' group:
 - 00/g1
 - 00/g2
 - 00/ra
 - 00/dec
 - 00/weight
 - 00/mag_i
 - 00/mag_r
 - 00/mag_z
 - 00/mag_err_i
 - 00/mag_err_r
 - 00/mag_err_z
Renaming these columns:
 - 00/g1 -> g1
 - 00/g2 -> g2
 - 00/ra -> ra
 - 00/dec -> dec
 - 00/weight -> weight
 - 00/mag_i -> mag_i
 - 00/mag_r -> mag_r
 - 00/mag_z -> mag_z
 - 00/mag_err_i -> mag_err_i
 - 00/mag_err_r -> mag_err_r
 - 00/mag_err_z -> mag_err_z
Process 0 collating bins: [0, 1, 2, 3, all]
Rank 0 processing data 0 - 82,200


In [10]:
path = "/global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5"

with h5py.File(path, "r") as f:
    zbin_groups = {}
    
    # Iterate through all subgroups under cluster_bin
    for name in f["cluster_bin"].keys():
        if name.startswith("bin_zbin_"):
            # Extract redshift bin index
            zbin = name.split("_richbin_")[0].replace("bin_zbin_", "")
            if zbin not in zbin_groups:
                zbin_groups[zbin] = []
            zbin_groups[zbin].append(f["cluster_bin"][name])
    
    # Now combine data for each zbin
    clusters_by_zbin = {}
    for zbin, groups in zbin_groups.items():
        combined = {}
        for g in groups:
            for key in g.keys():
                if key not in combined:
                    combined[key] = np.array(g[key])
                else:
                    combined[key] = np.concatenate([combined[key], np.array(g[key])])
        clusters_by_zbin[zbin] = combined

# Example: Access zbin_1 data
ra_z1 = clusters_by_zbin["1"]["ra"]
dec_z1 = clusters_by_zbin["1"]["dec"]
print(f"Redshift bin 1 has {len(ra_z1)} clusters.")

Redshift bin 1 has 28 clusters.


In [11]:
cluster_tomo = h5py.File(f"{my_txpipe_dir}/data/example/output_binning/cluster_catalog_tomography.hdf5")
#print(cluster_tomo["cluster_bin"][0])


with h5py.File("/global/homes/e/edujb/TXPipe/cluster_catalog_tomography.hdf5", "r") as f:
    def print_structure(name):
        print(name)
    f.visit(print_structure)
    print(f["cluster_bin/bin_zbin_0_richbin_0/redshift"][:])

cluster_bin
cluster_bin/bin_zbin_0_richbin_0
cluster_bin/bin_zbin_0_richbin_0/cluster_id
cluster_bin/bin_zbin_0_richbin_0/dec
cluster_bin/bin_zbin_0_richbin_0/ra
cluster_bin/bin_zbin_0_richbin_0/redshift
cluster_bin/bin_zbin_0_richbin_0/redshift_err
cluster_bin/bin_zbin_0_richbin_0/richness
cluster_bin/bin_zbin_0_richbin_0/richness_err
cluster_bin/bin_zbin_0_richbin_0/scaleval
cluster_bin/bin_zbin_0_richbin_1
cluster_bin/bin_zbin_0_richbin_1/cluster_id
cluster_bin/bin_zbin_0_richbin_1/dec
cluster_bin/bin_zbin_0_richbin_1/ra
cluster_bin/bin_zbin_0_richbin_1/redshift
cluster_bin/bin_zbin_0_richbin_1/redshift_err
cluster_bin/bin_zbin_0_richbin_1/richness
cluster_bin/bin_zbin_0_richbin_1/richness_err
cluster_bin/bin_zbin_0_richbin_1/scaleval
cluster_bin/bin_zbin_1_richbin_0
cluster_bin/bin_zbin_1_richbin_0/cluster_id
cluster_bin/bin_zbin_1_richbin_0/dec
cluster_bin/bin_zbin_1_richbin_0/ra
cluster_bin/bin_zbin_1_richbin_0/redshift
cluster_bin/bin_zbin_1_richbin_0/redshift_err
cluster_bin/bi

In [22]:
step3 = txpipe.extensions.cluster_counts.TXTwoPointRLens.make_stage(
    # This file is the input metadetect shear catalog
    binned_lens_catalog=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    # This is an input training set for the tomographic selection
    binned_shear_catalog=f"{my_txpipe_dir}data/example/output_shearcalib/binned_shear_catalog.hdf5",
    binned_random_catalog=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    patch_centers=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    tracer_metadata=f"{my_txpipe_dir}cluster_catalog_tomography.hdf5",
    use_randoms= False,
    output_dir=f"{my_txpipe_dir}data/example/output_binning/",
    metric="Euclidean",
    var_method="shot"
)

In [23]:
step3.run()
step3.finalize()

Running with 4 source bins and 6 lens bins
Running TreeCorr with metric "Euclidean"
DOING SHEAR-POS
Running 24 calculations: [(0, 0, 1), (0, 1, 1), (0, 2, 1), (0, 3, 1), (0, 4, 1), (0, 5, 1), (1, 0, 1), (1, 1, 1), (1, 2, 1), (1, 3, 1), (1, 4, 1), (1, 5, 1), (2, 0, 1), (2, 1, 1), (2, 2, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1), (3, 0, 1), (3, 1, 1), (3, 2, 1), (3, 3, 1), (3, 4, 1), (3, 5, 1)]
Rank 0 making patches for shear catalog bin 0
Patches already done for cache/patches/TXTwoPointRLensbinned_shear_catalog_4206620e42794f3f853ba2e1a68ff289/0
Rank 0 making patches for position catalog bin 0
Patches already done for cache/patches/TXTwoPointRLensbinned_lens_catalog_8de35c2d194c44b98232fe45139f53c4/0
Rank 0 making patches for shear catalog bin 1
Patches already done for cache/patches/TXTwoPointRLensbinned_shear_catalog_4206620e42794f3f853ba2e1a68ff289/1
Rank 0 making patches for position catalog bin 1
Patches already done for cache/patches/TXTwoPointRLensbinned_lens_catalog_8de35c2d194c44b98

NOTE/WARNING: Expected output file cluster_shear_catalogs.hdf5 was not generated.
